In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Plotting options
mpl.style.use('ggplot')
sns.set(style='whitegrid')

In [44]:
df= pd.read_csv(r'C:\Users\Alvi Mahmud\Desktop\BAN Sp20\Kag\train.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600000 entries, 0 to 599999
Data columns (total 25 columns):
id        600000 non-null int64
bin_0     582106 non-null float64
bin_1     581997 non-null float64
bin_2     582070 non-null float64
bin_3     581986 non-null object
bin_4     581953 non-null object
nom_0     581748 non-null object
nom_1     581844 non-null object
nom_2     581965 non-null object
nom_3     581879 non-null object
nom_4     581965 non-null object
nom_5     582222 non-null object
nom_6     581869 non-null object
nom_7     581997 non-null object
nom_8     582245 non-null object
nom_9     581927 non-null object
ord_0     581712 non-null float64
ord_1     581959 non-null object
ord_2     581925 non-null object
ord_3     582084 non-null object
ord_4     582070 non-null object
ord_5     582287 non-null object
day       582048 non-null float64
month     582012 non-null float64
target    600000 non-null int64
dtypes: float64(6), int64(2), object(17)
memory usage: 114.4

In [104]:
print(df.columns)
print(df.head(3))

#Mask the Null values to retain them during encoding
mask= df.isin(['nan'])

Index(['id', 'bin_0', 'bin_1', 'bin_2', 'bin_3', 'bin_4', 'nom_0', 'nom_1',
       'nom_2', 'nom_3', 'nom_4', 'nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9',
       'ord_0', 'ord_1', 'ord_2', 'ord_3', 'ord_4', 'ord_5', 'day', 'month',
       'target'],
      dtype='object')
   id  bin_0  bin_1  bin_2 bin_3 bin_4 nom_0      nom_1    nom_2   nom_3  ...  \
0   0    0.0    0.0    0.0     F     N   Red  Trapezoid  Hamster  Russia  ...   
1   1    1.0    1.0    0.0     F     Y   Red       Star  Axolotl     nan  ...   
2   2    0.0    1.0    0.0     F     N   Red        nan  Hamster  Canada  ...   

       nom_9 ord_0        ord_1     ord_2 ord_3 ord_4  ord_5  day month target  
0  02e7c8990   3.0  Contributor       Hot     c     U     Pw  6.0   3.0      0  
1  f37df64af   3.0  Grandmaster      Warm     e     X     pE  7.0   7.0      0  
2        nan   3.0          NaN  Freezing     n     P     eN  5.0   9.0      0  

[3 rows x 25 columns]


In [105]:
#Convert object to string
df[['bin_3','bin_4','nom_0', 'nom_1',
       'nom_2', 'nom_3', 'nom_4', 'nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9']]= df[['bin_3','bin_4','nom_0', 'nom_1',
       'nom_2', 'nom_3', 'nom_4', 'nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9']].astype('str')


In [124]:
#Encode all the nominal features and the two string binary feature

from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline


class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)
df_enc1= MultiColumnLabelEncoder(columns = [ 'bin_3','bin_4','nom_0', 'nom_1',
       'nom_2', 'nom_3', 'nom_4', 'nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9']).fit_transform(df)
df_enc1=df_enc1.where(~mask, other=np.nan)

In [125]:
df_enc1.bin_4.value_counts()

0.0    312344
1.0    269609
Name: bin_4, dtype: int64

In [126]:
#map low cardinality ordinal features
map_ord1 = {'Novice':1, 
            'Contributor':2, 
            'Expert':3, 
            'Master':4, 
            'Grandmaster':5}
df_enc1.ord_1 = df_enc1.ord_1.map(map_ord1)

map_ord2 = {'Freezing':1, 
            'Cold':3, 
            'Warm':5, 
            'Hot':7, 
            'Boiling Hot':9, 
            'Lava Hot':12}
df_enc1.ord_2 = df_enc1.ord_2.map(map_ord2)

In [127]:
#Encode high cardinality features
map_ord3 = {'a':1, 'b':2, 'c':3, 'd':4, 'e':5, 'f':6, 'g':7, 'h':8, 'i':9, 'j':10, 'k':11, 'l':12,'m':13,'n':14, '0':15}
df_enc1.ord_3 = df_enc1.ord_3.map(map_ord3)

map_ord4 = {'A':1, 'B':2, 'C':3, 'D':4, 'E':5, 'F':6, 'G':7, 'H':8, 'I':9, 'J':10, 'K':11, 'L':12,'M':13,'N':14, 'O':15,
           'P':16, 'Q':17, 'R':18, 'S':19, 'T':20, 'U':21, 'V':22, 'W':23, 'X':24, 'Y':25, 'Z':26}
df_enc1.ord_4 = df_enc1.ord_4.map(map_ord4)

In [128]:
import category_encoders as ce
df_enc1['ord_5_enc']=df_enc1['ord_5']
ce_ord = ce.OrdinalEncoder(cols = ['ord_5'])
df_enc1=ce_ord.fit_transform(df_enc1, df_enc1['ord_5_enc'])
df_enc1.ord_5_enc.value_counts()

4      17713
30     10562
38      9527
28      8654
12      5648
       ...  
147      189
190      124
188      120
189       91
191       87
Name: ord_5, Length: 191, dtype: int64

In [131]:
df_enc1.ord_5.value_counts()
df_encoded= df_enc1.drop('ord_5_enc', axis=1)

In [135]:
from fancyimpute import KNN
df_im= df_encoded.drop(['id','target'],axis=1).values
df_imputed = KNN(k=3).fit_transform(df_im)

MemoryError: Unable to allocate array with shape (600000, 600000) and data type float32

TypeError: 'module' object is not callable